In [1]:

# # Import required libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score

# # Load the training dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# # Check for missing values in the "Class" column
# print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# # Drop rows with missing values in the "Class" column
# train_data = train_data.dropna(subset=["Class"])

# # Map non-numeric labels to integers
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure there are no NaN values after mapping
# train_data = train_data.dropna(subset=["Class"])
# print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# # Split the training data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Ensure there are no NaN values in training and validation labels
# print("NaN in y_train:", pd.isna(y_train).sum())
# print("NaN in y_val:", pd.isna(y_val).sum())

# # Text Vectorization using TF-IDF
# vectorizer = TfidfVectorizer(max_features=5000)
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_val_tfidf = vectorizer.transform(X_val)

# # Initialize and Train the Logistic Regression Model
# model = LogisticRegression()
# model.fit(X_train_tfidf, y_train)

# # Evaluate on the validation set
# y_val_pred = model.predict(X_val_tfidf)
# print("Validation Classification Report:\n", classification_report(y_val, y_val_pred))
# print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))

# # Load the test dataset
# test_data = pd.read_csv("/kaggle/input/test-data/test_predictions.csv")

# # Vectorize the test data
# X_test_tfidf = vectorizer.transform(test_data["Text"])

# # Make predictions on the test dataset
# test_predictions = model.predict(X_test_tfidf)

# # Convert predictions back to original label names
# reverse_label_mapping = {v: k for k, v in label_mapping.items()}
# test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# # Save predictions to a CSV file
# output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
# output.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


Number of missing values in 'Class' before cleaning: 0
Number of missing values in 'Class' after mapping and cleaning: 0
NaN in y_train: 0
NaN in y_val: 0
Validation Classification Report:
               precision    recall  f1-score   support

         0.0       0.67      0.72      0.70       281
         1.0       0.70      0.64      0.67       277

    accuracy                           0.68       558
   macro avg       0.68      0.68      0.68       558
weighted avg       0.68      0.68      0.68       558

Validation Accuracy: 0.6827956989247311
Predictions saved to 'test_predictions.csv'.


In [2]:
pip install transformers datasets


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install evaluate


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [8]:
# # Import required libraries
# import pandas as pd
# from datasets import Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#     }

# # Define the Trainer arguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",               # Evaluate every epoch
#     save_strategy="epoch",               # Save every epoch
#     learning_rate=5e-5,                  # Learning rate
#     per_device_train_batch_size=16,      # Batch size for training
#     per_device_eval_batch_size=16,       # Batch size for evaluation
#     num_train_epochs=3,                  # Number of epochs
#     weight_decay=0.01,                   # Weight decay
#     logging_dir="./logs",                # Logging directory
#     logging_steps=10,                    # Log every 10 steps
#     report_to="none",                    # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,  # Add validation dataset
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,  # Add the metrics function
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions
# test_data["Predicted_Class"] = pred_labels
# test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
# test_data.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.250400,0.239480,0.483871,0.483871,1.000000
2,0.202600,0.208212,0.483871,0.483871,1.000000
3,0.176800,0.185809,0.483871,0.483871,1.000000


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predictions saved to 'test_predictions.csv'.


In [4]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
)

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,  # F1 Score
    }

# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=7,                   # Number of epochs
    weight_decay=0.003,                    # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add the metrics function
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_distilbert.csv", index=False)
print("Predictions saved to 'test_predictions_distilbert.csv'.") 


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather a

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.215100,0.189506,0.496416,0.496416,1.000000,0.663473
2,0.198000,0.179258,0.496416,0.496416,1.000000,0.663473
3,0.155500,0.167940,0.602151,0.555781,0.989170,0.711688
4,0.084000,0.179847,0.645161,0.586433,0.967509,0.730245
5,0.052500,0.195791,0.664875,0.605140,0.935018,0.734752
6,0.034700,0.196351,0.627240,0.575824,0.945848,0.715847
7,0.027500,0.197740,0.616487,0.569536,0.931408,0.706849


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predictions saved to 'test_predictions_distilbert.csv'.


In [11]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=7,                   # Number of epochs
    weight_decay=0.003,                   # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions with only "Id" and "Labels"
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_bert.csv", index=False)
print("Predictions saved to 'test_predictions_bert.csv'.") 

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-bac636967b73>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.203900,0.218733,0.483871,0.483871,1.000000,0.652174
2,0.172200,0.180378,0.505376,0.494505,1.000000,0.661765
3,0.126100,0.204632,0.526882,0.505618,1.000000,0.671642
4,0.104000,0.203772,0.709677,0.643617,0.896296,0.749226
5,0.057600,0.229401,0.706093,0.634518,0.925926,0.753012
6,0.056000,0.222159,0.483871,0.483871,1.000000,0.652174
7,0.020800,0.237181,0.702509,0.639785,0.881481,0.741433


<ipython-input-11-bac636967b73>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
<ipython-input-11-bac636967b73>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gath

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predictions saved to 'test_predictions_bert.csv'.


In [11]:
# import pandas as pd

# # Load the predictions
# test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_predictions[['id', 'Predicted_Class']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")


EmptyDataError: No columns to parse from file

In [10]:
# import pandas as pd
# from datasets import Dataset
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
#     f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,  # F1 Score
#     }

# # Define the TrainingArguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",                # Evaluate every epoch
#     save_strategy="epoch",                # Save every epoch
#     learning_rate=5e-5,                   # Learning rate (try adjusting this)
#     per_device_train_batch_size=16,       # Batch size for training
#     per_device_eval_batch_size=16,        # Batch size for evaluation
#     num_train_epochs=7,                   # Number of epochs
#     weight_decay=0.003,                    # Weight decay
#     logging_dir="./logs",                 # Logging directory
#     logging_steps=10,                     # Log every 10 steps
#     report_to="none",                     # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,  # Add validation dataset
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,  # Add the metrics function
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions
# test_data["Labels"] = pred_labels
# test_data["Labels"] = test_data["Labels"].map({0: "Non-Abusive", 1: "Abusive"})
# #test_data.to_csv("test_predictions_bert.csv", index=False)
# #print("Predictions saved to 'test_predictions_bert.csv'.") 
# # Load the predictions
# #test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_data[['id', 'Labels']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels1.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-a78e34157025>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.195500,0.158613,0.483871,0.483871,1.000000,0.652174
2,0.165400,0.179616,0.483871,0.483871,1.000000,0.652174
3,0.107900,0.186692,0.483871,0.483871,1.000000,0.652174
4,0.076500,0.207696,0.491039,0.487365,1.000000,0.655340
5,0.063500,0.200183,0.483871,0.483871,1.000000,0.652174
6,0.039900,0.229876,0.483871,0.483871,1.000000,0.652174
7,0.014900,0.234219,0.548387,0.517510,0.985185,0.678571


<ipython-input-10-a78e34157025>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
<ipython-input-10-a78e34157025>:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gath

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.


In [10]:
# import pandas as pd

# # Load the predictions
# test_predictions = pd.read_csv("test_predictions_bert.csv")

# # Create the file with only 'id' and 'Predicted_Class'
# prediction_file = test_predictions[['id', 'Predicted_Class']]

# # Save the file
# prediction_file.to_csv("test_predictions_bert_only_id_labels.csv", index=False)

# print("Prediction file with 'id' and 'Predicted_Class' saved as 'test_predictions_bert_only_id_labels.csv'.")


EmptyDataError: No columns to parse from file

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Load the dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Map the labels
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure no missing values
train_data = train_data.dropna(subset=["Class"])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.1, random_state=42
)

# Initialize the tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

# Tokenize the dataset
def preprocess_function(texts, labels):
    tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = labels
    return tokenized

train_encodings = preprocess_function(train_texts, list(train_labels))
val_encodings = preprocess_function(val_texts, list(val_labels))

# Convert tokenized data to PyTorch datasets
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

# Initialize the model
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)

# Define the compute_metrics function
def compute_metrics(pred):
    predictions, labels = pred
    preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
    labels = torch.tensor(labels).int()
    
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0  # F1 Score
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,  # F1 Score
    }

# Define the Trainer arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    eval_strategy="epoch",                # Evaluate every epoch
    save_strategy="epoch",                # Save every epoch
    learning_rate=5e-5,                   # Learning rate
    per_device_train_batch_size=16,       # Batch size for training
    per_device_eval_batch_size=16,        # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    weight_decay=0.003,                    # Weight decay
    logging_dir="./logs",                 # Logging directory
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable W&B logging
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add the metrics function
)

# Train the model
trainer.train()

# Make predictions on the test set
test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
test_dataset = CustomDataset(test_encodings)

predictions = trainer.predict(test_dataset)
preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
pred_labels = (preds > 0.5).astype(int).flatten()

# Save predictions
test_data["Predicted_Class"] = pred_labels
test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
test_data.to_csv("test_predictions_xlmr.csv", index=False)
print("Predictions saved to 'test_predictions_xlmr.csv'.")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.260800,0.225414,0.483871,0.483871,1.000000,0.652174
2,0.258700,0.249904,0.483871,0.483871,1.000000,0.652174
3,0.260100,0.249768,0.483871,0.483871,1.000000,0.652174


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predictions saved to 'test_predictions_xlmr.csv'.


In [2]:
# # Import required libraries
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import classification_report, accuracy_score

# # Load the training dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# # Check for missing values in the "Class" column
# print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# # Drop rows with missing values in the "Class" column
# train_data = train_data.dropna(subset=["Class"])

# # Map non-numeric labels to integers
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure there are no NaN values after mapping
# train_data = train_data.dropna(subset=["Class"])
# print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# # Split the training data into train and validation sets
# X_train, X_val, y_train, y_val = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Ensure there are no NaN values in training and validation labels
# print("NaN in y_train:", pd.isna(y_train).sum())
# print("NaN in y_val:", pd.isna(y_val).sum())

# # Define a function to train and evaluate models
# def train_and_evaluate(models, feature_extractor, feature_name):
#     # Transform text data using the feature extractor
#     X_train_features = feature_extractor.fit_transform(X_train)
#     X_val_features = feature_extractor.transform(X_val)
    
#     # Iterate over the models and train them
#     for model_name, model in models.items():
#         print(f"\nTraining {model_name} with {feature_name} features...")
#         model.fit(X_train_features, y_train)
#         y_val_pred = model.predict(X_val_features)
        
#         # Evaluate the model
#         print(f"\nValidation Results for {model_name} ({feature_name}):")
#         print("Classification Report:\n", classification_report(y_val, y_val_pred))
#         print("Accuracy:", accuracy_score(y_val, y_val_pred))

# # Define models to train
# models = {
#     "Logistic Regression": LogisticRegression(),
#     "SVM": SVC(),
#     "Random Forest": RandomForestClassifier(),
#     "Multinomial Naive Bayes": MultinomialNB()
# }

# # Evaluate with TF-IDF features
# print("\nUsing TF-IDF features:")
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)
# train_and_evaluate(models, tfidf_vectorizer, "TF-IDF")

# # Evaluate with Bag of Words (BoW) features
# print("\nUsing Bag of Words (BoW) features:")
# bow_vectorizer = CountVectorizer(max_features=5000)
# train_and_evaluate(models, bow_vectorizer, "BoW")

# # Load the test dataset
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Transform test data using the TF-IDF vectorizer
# X_test_tfidf = tfidf_vectorizer.transform(test_data["Text"])

# # Make predictions on the test dataset using the best-performing model (e.g., Logistic Regression)
# best_model = LogisticRegression()  # Replace with your chosen model if needed
# best_model.fit(tfidf_vectorizer.fit_transform(X_train), y_train)  # Retrain on the full training data
# test_predictions = best_model.predict(X_test_tfidf)

# # Convert predictions back to original label names
# reverse_label_mapping = {v: k for k, v in label_mapping.items()}
# test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# # Save predictions to a CSV file
# output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
# output.to_csv("test_predictions.csv", index=False)
# print("Predictions saved to 'test_predictions.csv'.")


Number of missing values in 'Class' before cleaning: 0
Number of missing values in 'Class' after mapping and cleaning: 0
NaN in y_train: 0
NaN in y_val: 0

Using TF-IDF features:

Training Logistic Regression with TF-IDF features...

Validation Results for Logistic Regression (TF-IDF):
Classification Report:
               precision    recall  f1-score   support

         0.0       0.67      0.72      0.70       281
         1.0       0.70      0.64      0.67       277

    accuracy                           0.68       558
   macro avg       0.68      0.68      0.68       558
weighted avg       0.68      0.68      0.68       558

Accuracy: 0.6827956989247311

Training SVM with TF-IDF features...

Validation Results for SVM (TF-IDF):
Classification Report:
               precision    recall  f1-score   support

         0.0       0.66      0.73      0.69       281
         1.0       0.69      0.62      0.66       277

    accuracy                           0.68       558
   macro avg   

In [3]:
# Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Load the training dataset
train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")

# Check for missing values in the "Class" column
print("Number of missing values in 'Class' before cleaning:", train_data["Class"].isna().sum())

# Drop rows with missing values in the "Class" column
train_data = train_data.dropna(subset=["Class"])

# Map non-numeric labels to integers
label_mapping = {"Non-Abusive": 0, "Abusive": 1}
train_data["Class"] = train_data["Class"].map(label_mapping)

# Ensure there are no NaN values after mapping
train_data = train_data.dropna(subset=["Class"])
print("Number of missing values in 'Class' after mapping and cleaning:", train_data["Class"].isna().sum())

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
)

# Ensure there are no NaN values in training and validation labels
print("NaN in y_train:", pd.isna(y_train).sum())
print("NaN in y_val:", pd.isna(y_val).sum())

# Define a function to train and evaluate models
def train_and_evaluate(models, feature_extractor, feature_name):
    # Transform text data using the feature extractor
    X_train_features = feature_extractor.fit_transform(X_train)
    X_val_features = feature_extractor.transform(X_val)
    
    # Iterate over the models and train them
    for model_name, model in models.items():
        print(f"\nTraining {model_name} with {feature_name} features...")
        model.fit(X_train_features, y_train)
        y_val_pred = model.predict(X_val_features)
        
        # Evaluate the model
        print(f"\nValidation Results for {model_name} ({feature_name}):")
        print("Classification Report:\n", classification_report(y_val, y_val_pred, digits=4))
        accuracy = accuracy_score(y_val, y_val_pred)
        print(f"Accuracy: {accuracy:.4f}")

# Define models to train
models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Multinomial Naive Bayes": MultinomialNB()
}

# Evaluate with TF-IDF features
print("\nUsing TF-IDF features:")
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_and_evaluate(models, tfidf_vectorizer, "TF-IDF")

# Evaluate with Bag of Words (BoW) features
print("\nUsing Bag of Words (BoW) features:")
bow_vectorizer = CountVectorizer(max_features=5000)
train_and_evaluate(models, bow_vectorizer, "BoW")

# Load the test dataset
test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# Transform test data using the TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(test_data["Text"])

# Make predictions on the test dataset using the best-performing model (e.g., Logistic Regression)
best_model = LogisticRegression()  # Replace with your chosen model if needed
best_model.fit(tfidf_vectorizer.fit_transform(X_train), y_train)  # Retrain on the full training data
test_predictions = best_model.predict(X_test_tfidf)

# Convert predictions back to original label names
reverse_label_mapping = {v: k for k, v in label_mapping.items()}
test_predictions_labels = [reverse_label_mapping[pred] for pred in test_predictions]

# Save predictions to a CSV file
output = pd.DataFrame({"Text": test_data["Text"], "Predicted_Class": test_predictions_labels})
output.to_csv("test_predictions.csv", index=False)
print("Predictions saved to 'test_predictions.csv'.")


Number of missing values in 'Class' before cleaning: 0
Number of missing values in 'Class' after mapping and cleaning: 0
NaN in y_train: 0
NaN in y_val: 0

Using TF-IDF features:

Training Logistic Regression with TF-IDF features...

Validation Results for Logistic Regression (TF-IDF):
Classification Report:
               precision    recall  f1-score   support

         0.0     0.6722    0.7224    0.6964       281
         1.0     0.6953    0.6426    0.6679       277

    accuracy                         0.6828       558
   macro avg     0.6837    0.6825    0.6822       558
weighted avg     0.6837    0.6828    0.6823       558

Accuracy: 0.6828

Training SVM with TF-IDF features...

Validation Results for SVM (TF-IDF):
Classification Report:
               precision    recall  f1-score   support

         0.0     0.6613    0.7295    0.6937       281
         1.0     0.6935    0.6209    0.6552       277

    accuracy                         0.6756       558
   macro avg     0.6774    

In [5]:
# import pandas as pd
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score

# # Load the dataset
# train_data = pd.read_csv("/kaggle/input/train-data/AWT_train.csv")
# test_data = pd.read_csv("/kaggle/input/test1-1-data/AWT_test_without_labels.csv")

# # Map the labels
# label_mapping = {"Non-Abusive": 0, "Abusive": 1}
# train_data["Class"] = train_data["Class"].map(label_mapping)

# # Ensure no missing values
# train_data = train_data.dropna(subset=["Class"])

# # Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     train_data["Text"], train_data["Class"], test_size=0.2, random_state=42
# )

# # Initialize the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize the dataset
# def preprocess_function(texts, labels):
#     tokenized = tokenizer(list(texts), padding="max_length", truncation=True, max_length=128, return_tensors="pt")
#     tokenized["labels"] = labels
#     return tokenized

# train_encodings = preprocess_function(train_texts, list(train_labels))
# val_encodings = preprocess_function(val_texts, list(val_labels))

# # Convert tokenized data to PyTorch datasets
# class CustomDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __len__(self):
#         return len(self.encodings["input_ids"])

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# train_dataset = CustomDataset(train_encodings)
# val_dataset = CustomDataset(val_encodings)

# # Initialize the model
# model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)

# # Define the compute_metrics function
# def compute_metrics(pred):
#     predictions, labels = pred
#     preds = (torch.sigmoid(torch.tensor(predictions)) > 0.5).int()
#     labels = torch.tensor(labels).int()
    
#     accuracy = accuracy_score(labels, preds)
#     precision = precision_score(labels, preds)
#     recall = recall_score(labels, preds)
#     f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
#     return {
#         "accuracy": accuracy,
#         "precision": precision,
#         "recall": recall,
#         "f1": f1,
#     }

# # Define the TrainingArguments
# training_args = TrainingArguments(
#     output_dir="./results",               # Output directory
#     eval_strategy="epoch",                # Evaluate every epoch
#     save_strategy="epoch",                # Save every epoch
#     learning_rate=5e-5,                   # Learning rate
#     per_device_train_batch_size=16,       # Batch size for training
#     per_device_eval_batch_size=16,        # Batch size for evaluation
#     num_train_epochs=7,                   # Number of epochs
#     weight_decay=0.003,                   # Weight decay
#     logging_dir="./logs",                 # Logging directory
#     logging_steps=10,                     # Log every 10 steps
#     report_to="none",                     # Disable W&B logging
# )

# # Define the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

# # Train the model
# trainer.train()

# # Make predictions on the test set
# test_encodings = preprocess_function(test_data["Text"], [0] * len(test_data))
# test_dataset = CustomDataset(test_encodings)

# predictions = trainer.predict(test_dataset)
# preds = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
# pred_labels = (preds > 0.5).astype(int).flatten()

# # Save predictions with only "Id" and "Labels"
# test_data["Predicted_Class"] = pred_labels
# test_data["Predicted_Class"] = test_data["Predicted_Class"].map({0: "Non-Abusive", 1: "Abusive"})
# test_data.to_csv("test_predictions_bert.csv", index=False)
# print("Predictions saved to 'test_predictions_bert.csv'.") 

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-5-53ce25c34657>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.261500,0.250011,0.496416,0.496416,1.000000,0.663473
2,0.261800,0.256118,0.496416,0.496416,1.000000,0.663473
3,0.257800,0.256510,0.496416,0.496416,1.000000,0.663473
4,0.258900,0.261439,0.496416,0.496416,1.000000,0.663473
5,0.253500,0.249988,0.496416,0.496416,1.000000,0.663473
6,0.251800,0.250443,0.496416,0.496416,1.000000,0.663473
7,0.252300,0.250063,0.496416,0.496416,1.000000,0.663473


<ipython-input-5-53ce25c34657>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
<ipython-input-5-53ce25c34657>:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Predictions saved to 'test_predictions_bert.csv'.
